# Monotonically decreasing SoH
The goal of this notebook is to study the results that has been put in the database.

## Setup

### Imports

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from core.stats_utils import *
from transform.raw_results.config import *
from transform.processed_results.config import *

from transform.processed_results.main import get_processed_results, make_soh_presentable_per_vehicle

### Data extraction

In [ ]:
results = get_processed_results("tesla")

### Visualization
Let's visualize the results

In [ ]:
px.scatter(results, 
           x="odometer", 
           y="soh", 
           color="model", 
           hover_data=["vin"],
           trendline="ols",
           title="Original SoH results")

In [ ]:
# nbr de vin avec un soh calculé
results['vin'].nunique()

### compute data processed

In [ ]:
from logging import getLogger

logger = getLogger("eda.results.monotonically_decreasing_soh")

# Application du traitement sur le DataFrame original
ts_processed = (
    results
    # 3. Appliquer le traitement de SoH par véhicule
    .groupby('vin', observed=True)
    .apply(make_soh_presentable_per_vehicle, include_groups=False)
    # 1. D'abord, arrondir les dates à la semaine 
    .assign(date=lambda df: df["date"].dt.floor(UPDATE_FREQUENCY))
    # 2. Grouper par vin et date pour avoir une valeur hebdomadaire
    .groupby(["vin", "date"], observed=True,)
    .agg({
        "odometer": "last",    
        "soh": "median",
        "model": "first",
    })
    .pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger).reset_index()
)

In [ ]:
px.line(ts_processed.sort_values(["vin", "odometer"]).dropna(subset=["soh", "odometer"]), x="odometer", y="soh", color="vin", title="Processed SoH results")

## Check if every vin soh are decreasing 

In [ ]:
def soh_is_monotonically_decreasing(df):
    return df.set_index("date")["soh"].is_monotonic_decreasing

In [ ]:
montonic_mask_per_vin = (
    ts_processed.dropna(subset=['soh'])
    .groupby("vin", observed=True)
    .apply(soh_is_monotonically_decreasing, include_groups=False)
    .to_frame("monotonically_decreasing")
    .reset_index(drop=False)
)

In [ ]:
value_counts_per_vin = {
    "normalized": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=True),
    "absolute": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=False)
}
pd.concat(value_counts_per_vin, axis=1, keys=value_counts_per_vin.keys(), names=["value_counts_type"])